# Predicting how points end in tennis (CrowdAnalytix)

## Keras 3-layer Artificial Neural Network

## Motivation
Tennis, one of the most popular professional sports around the world, still uses manual coding of point outcomes.  This is not only labor-intensive but it also raises concerns that outcome categories may not always be consistent from one coder to the next. The purpose of this contest is to find a better approach. 

## Point Endings
Every tennis match is made up of a sequence of points. A point begins with a serve and players exchange shots until a player makes an error or is unable to return a shot in play. 

Traditionally, the shot ending a point in tennis has been had been described in one of three mutually exclusive ways: a winner, an unforced error, or a forced error. A winner is a shot that was in play, not touched by the opponent, and ends with the point going to the player who made the shot. The other two categories are two distinct types of errors where both end with the point going to the player who did not make the shot. The distinction between an unforced and forced error is based on the nature of the incoming shot and a judgment about whether the shot was playable or not. As you can imagine, this distinction is not a perfect science.  

## Outcome Coding
Point endings give us insight into player performance. For this reason, accurate statistics about point outcomes are essential to the sport. At professional tennis tournaments, human coders are trained to label and document outcomes during matches. This is the primary way that the sport gathers information about winners and errors. 

## Tracking Data
The adoption of the player challenge system in the mid-2000s has lead to the use of multi-camera tracking systems for the majority of top professional matches. These tracking systems monitor the 3D coordinates of the ball position and 2D coordinates of the player position throughout a match. The richness of these data hold considerable promise for addressing many challenging questions in the sport.

## Objective

The objective of this contest is as follows:

* Predict how a point ends in tennis using modern tracking data.

In [25]:
import warnings; warnings.simplefilter('ignore')

from time import time
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard
from keras.wrappers.scikit_learn import KerasClassifier

## Data

In [26]:
# Train data.
df_mens = pd.read_csv('data/mens_train_file.csv', sep=',',header=0)
df_womens = pd.read_csv('data/womens_train_file.csv', sep=',',header=0)
frames = [df_mens, df_womens]
df = pd.concat(frames)

# Submission data.
df_mens_test = pd.read_csv('data/mens_test_file.csv', sep=',',header=0)
df_womens_test = pd.read_csv('data/womens_test_file.csv', sep=',',header=0)
frames = [df_mens_test, df_womens_test]
df_test = pd.concat(frames)
df_test['submission_id'] = df_test['id'].map(str) + '_' + df_test['gender'].map(str)
df_submission = pd.read_csv('data/AUS_SubmissionFormat.csv', sep=',',header=0)
df_test = pd.merge(df_submission, df_test, how='outer', on=['submission_id', 'submission_id'])
df_test.drop(['submission_id', 'train_x', 'UE', 'FE', 'W'], axis=1, inplace=True)

In [27]:
print(df.head())

   rally  serve hitpoint      speed  net.clearance  distance.from.sideline  \
0      4      1        B  35.515042      -0.021725                3.474766   
1      4      2        B  33.382640       1.114202                2.540801   
2     23      1        B  22.316690      -0.254046                3.533166   
3      9      1        F  36.837309       0.766694                0.586885   
4      4      1        B  35.544208       0.116162                0.918725   

      depth  outside.sideline  outside.baseline  player.distance.travelled  \
0  6.797621             False             False                   1.467570   
1  2.608708             False              True                   2.311931   
2  9.435749             False             False                   3.903728   
3  3.342180              True             False                   0.583745   
4  5.499119             False             False                   2.333456   

    ...    opponent.depth  opponent.distance.from.center  same

In [54]:
X = df.iloc[:, 1:24].values
Y = df.iloc[:, 26].values
X_pred = df_test.iloc[:, 1:24].values


### Pre-processing

In [55]:
# Encoding categorical data.
labelEncoder = LabelEncoder()
for col in [1,6,7,19,20,22]:
    X[:, col] = labelEncoder.fit_transform(X[:, col])
    X_pred[:, col] = labelEncoder.fit_transform(X_pred[:, col])

# Categorical representation: ['FE', 'UE', 'W']
Y = keras.utils.to_categorical(labelEncoder.fit_transform(Y), num_classes=3)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle=True)

# Feature Scaling.
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_pred = sc.transform(X_pred)


In [56]:
# Check shapes.
print("X_train: ", X_train.shape)
print("Y_train: ", Y_train.shape)
print("X_test: ", X_test.shape)
print("Y_test: ", Y_test.shape)


X_train:  (8000, 23)
Y_train:  (8000, 3)
X_test:  (2000, 23)
Y_test:  (2000, 3)


### Model

In [57]:
def classifier():

    model = Sequential()

    model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    return model

![title](img/graph.png)

### Train

In [62]:

model = classifier()

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(X_train, Y_train,
          epochs=120,
          batch_size=25,
          callbacks=[tensorboard])


Epoch 1/120
  25/8000 [..............................] - ETA: 52s - loss: 1.1919 - acc: 0.3200

1050/8000 [==>...........................] - ETA: 1s - loss: 1.0530 - acc: 0.4714 

2125/8000 [======>.......................] - ETA: 0s - loss: 0.9873 - acc: 0.5087

3200/8000 [===========>..................] - ETA: 0s - loss: 0.9312 - acc: 0.5513

4025/8000 [==============>...............] - ETA: 0s - loss: 0.8888 - acc: 0.5781

...

4825/8000 [=================>............] - ETA: 0s - loss: 0.2751 - acc: 0.8991

5850/8000 [====================>.........] - ETA: 0s - loss: 0.2794 - acc: 0.8968

6650/8000 [=======================>......] - ETA: 0s - loss: 0.2738 - acc: 0.8989

7575/8000 [===========================>..] - ETA: 0s - loss: 0.2774 - acc: 0.8966

8000/8000 [==============================] - 0s 54us/step - loss: 0.2769 - acc: 0.8970


![acc](img/acc.png)

### Evaluation

In [63]:
print('Testing:')
score = model.evaluate(X_test, Y_test)
print(model.metrics_names[0], ': ', score[0], '\n', model.metrics_names[1], ': ',score[1])


Testing:


  32/2000 [..............................] - ETA: 18s

1024/2000 [==============>...............] - ETA: 0s 

2000/2000 [==============================] - 0s 199us/step


loss :  0.36042029535770415 
 acc :  0.8645
